# **SUBMISSION 1: Email Spam Machine Learning Pipeline** 

Nama: `Shendi Teuku Maulana Efendi` <br>
Username Dicoding: `shendyeff`

## **Import Library**

In [48]:
# Library dasar
import os
import numpy as np
import pandas as pd

# TensorFlow & TFX
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
from tfx.components import (
    CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator,
    Transform, Trainer, Evaluator, Pusher, Tuner
)
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2, trainer_pb2
from tfx.types import standard_artifacts

# Model Development
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
from tensorflow.keras import layers, models, optimizers

# Resolver for resolving inputs in pipelines
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy

# TFX standard artifact types
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

# Requests and PrettyPrinter for external data handling and debugging
import requests
from pprint import PrettyPrinter

# Logging
from absl import logging
logging.set_verbosity(logging.INFO)

## **Set Variable**

In [2]:
# Define pipeline and schema names
PIPELINE_NAME = "email-spam-pipeline"  # Name of the main pipeline
SCHEMA_PIPELINE_NAME = "email-spam-tfdv-schema"  # Name of the schema pipeline
MODEL_NAME = "email-spam-model" # Name of the saved model

# Directory for storing generated artifacts
PIPELINE_ROOT = os.path.join('shendi_teuku_maulana_efendi-pipeline', PIPELINE_NAME)  # Root directory for pipeline artifacts

# Path to SQLite DB file for MLMD storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')  # Path to metadata database

# Output directory for exporting trained models
SERVING_MODEL_DIR = os.path.join('serving_model_dir', MODEL_NAME)  # Directory for saving trained models

# Pipeline inputs
DATA_ROOT = "data"  # Root directory for input data
TRANSFORM_MODULE_FILE = "modules/email_spam_transform.py"  # Transformation logic module
TRAINER_MODULE_FILE = "modules/email_spam_trainer.py"  # Model training logic module
TUNER_MODULE_FILE = "modules/email_spam_tuner.py"  # Hyperparameter tuning module

In [3]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## **Processing Dataset with Pandas**

In [4]:
df = pd.read_csv("data\email_spam.csv", encoding='latin-1')
df.head()

,is_spam,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.rename(columns={'Category': 'is_spam', 'Message': 'message'}, inplace=True)
df.head()

,is_spam,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.is_spam.value_counts()

0    4825
1     747
Name: is_spam, dtype: int64

In [7]:
df['is_spam'] = df['is_spam'].replace({'ham': 0, 'spam': 1})
df.head()

,is_spam,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.to_csv("data/email_spam.csv", index=False)
df.head()

,is_spam,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## **Membangun Pipeline TFX**
Dalam proyek machine learning, pipeline adalah alur yang menghubungkan berbagai tahap mulai dari pemrosesan data hingga evaluasi model. Dengan TensorFlow Extended (TFX), dapat mengotomatisasi seluruh proses tersebut dengan menggunakan komponen-komponen utama. Di bawah ini adalah komponen-komponen yang membentuk sebuah pipeline TFX, yang meliputi `CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Tuner, Trainer, Evaluator, dan Pusher`.

Dengan pendekatan modular ini akan menyesuaikan setiap bagian sesuai dengan kebutuhan proyek. Selain itu, juga akan menunjukkan penggunaan magic command seperti `%%writefile` yang memungkinkan pembuatan modul Python secara otomatis, memudahkan pembuatan modul khusus untuk komponen-komponen tertentu seperti `Transform, Trainer, dan Tuner`.

## **Data Ingestion**
Proses data ingestion di pipeline TFX dilakukan menggunakan komponen `CsvExampleGen`, yang berfungsi untuk membaca file CSV dan mengonversinya menjadi format `TFRecord` yang lebih efisien. Komponen ini juga memungkinkan pembagian data menjadi beberapa subset, seperti training dan evaluasi, melalui konfigurasi split. Setelah pemrosesan, output berupa `TFRecord` siap digunakan oleh komponen berikutnya dalam pipeline, seperti `StatisticsGen` atau `SchemaGen`, untuk analisis lebih lanjut. Dengan `CsvExampleGen`, pembagian dan pemrosesan data menjadi lebih otomatis dan efisien.

### **CsvExampleGen**
`CsvExampleGen` digunakan untuk melakukan data ingestion dari file CSV. Parameter `input_base` berisi direktori sumber data, yang dalam hal ini adalah DATA_ROOT. Konfigurasi pembagian data diatur melalui `output_config`, yang mendefinisikan dua subset: satu untuk training (`train`) dengan 8 hash bucket dan satu lagi untuk evaluasi (`eval`) dengan 2 hash bucket. Pembagian ini memastikan data terdistribusi secara acak antara kedua subset. Hasil outputnya adalah data yang telah diproses dalam format TFRecord dan siap untuk digunakan di komponen pipeline berikutnya.<br><br>
`Hash bucket`: untuk membagi data secara acak ke dalam beberapa kelompok atau subset

In [9]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [10]:
interactive_context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data data\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 45
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## **Data Validation**
Untuk bagian `Data Validation` dalam pipeline `TFX`, proses ini memastikan bahwa data yang digunakan untuk pelatihan model memenuhi kriteria yang diharapkan dan tidak mengandung masalah atau anomali yang bisa mempengaruhi hasil model. Ada beberapa langkah penting dalam proses validasi data diantaranya ada dibawah ini:
- `StatisticsGen`: Menghitung statistik dasar untuk memahami karakteristik data.
- `SchemaGen`: Membuat skema untuk mendefinisikan tipe dan batasan data.
- `ExampleValidator`: Mendeteksi anomali dan memastikan data valid sesuai skema.

### **StatisticGen**
Dengan menggunakan komponen StatisticsGen, pipeline ini menghitung statistik dasar seperti jumlah nilai, rata-rata, deviasi standar, nilai minimum, maksimum, dan distribusi dari setiap fitur dalam dataset. Hal ini penting untuk mengidentifikasi apakah ada data yang tidak konsisten atau tidak normal, seperti nilai yang hilang atau outlier.

In [11]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

INFO:absl:Excluding no splits because exclude_splits is not set.


In [12]:
interactive_context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\StatisticsGen\statistics\46\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\StatisticsGen\statistics\46\Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 46
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [13]:
interactive_context.show(statistics_gen.outputs["statistics"])

### **DataSchema**
`Data Schema` adalah definisi yang menggambarkan struktur data yang diharapkan. Pada langkah ini, komponen SchemaGen digunakan untuk menghasilkan skema data berdasarkan statistik yang telah dihitung. Skema ini mendefinisikan tipe data untuk setiap fitur, apakah nilai fitur tersebut numerik, kategori, atau lainnya, serta apakah fitur tersebut boleh bernilai null atau tidak.

In [14]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)

INFO:absl:Excluding no splits because exclude_splits is not set.


In [15]:
interactive_context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\SchemaGen\schema\47\schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 47
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'is_spam',INT,required,,-
'message',BYTES,required,,-


Skema yang dihasilkan oleh `SchemaGen` menunjukkan dua fitur utama dalam dataset: `Category` dan `Message`. Fitur `Category` bertipe STRING dan memiliki nilai yang diperlukan, dengan dua kategori yang valid yaitu `ham` dan `spam`. Fitur Message bertipe BYTES dan juga bersifat wajib, tetapi tidak memiliki nilai terbatas. Skema ini memastikan bahwa data yang digunakan sesuai dengan struktur yang diharapkan.

### **ExampleValidator**
Digunakan untuk mendeteksi anomali atau nilai-nilai yang tidak biasa dalam data yang bisa merusak kualitas model. Misalnya, data yang sangat tidak normal (outliers) atau nilai yang tidak sesuai dengan distribusi data yang diharapkan.

Dengan menggunakan komponen ExampleValidator, kita bisa melakukan validasi terhadap data berdasarkan skema yang telah didefinisikan dan mendeteksi apakah ada contoh data yang melanggar aturan tersebut. Misalnya, contoh data yang memiliki nilai yang terlalu besar, nilai null pada kolom yang tidak diperbolehkan, atau nilai yang berada di luar rentang yang diharapkan.

In [17]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

INFO:absl:Excluding no splits because exclude_splits is not set.


In [18]:
interactive_context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\ExampleValidator\anomalies\48\Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\ExampleValidator\anomalies\48\Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 48
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [19]:
interactive_context.show(example_validator.outputs['anomalies'])

Hasil dari `ExampleValidator` menunjukkan bahwa tidak ditemukan anomali pada kedua split dataset, yaitu `train` dan `eval`. Ini menandakan bahwa data yang digunakan untuk pelatihan dan evaluasi sudah valid dan sesuai dengan skema yang ditentukan.

## **Data Processing**

Untuk memulai Data Processing menggunakan TFX, pertama-tama perlu membuat file modul transformasi, yang biasa disebut transform module file. Modul ini akan berisi logika transformasi data menggunakan TensorFlow Transform (TFT). Di dalam file ini, kita mendefinisikan fungsi untuk menyesuaikan data sesuai kebutuhan model, seperti normalisasi atau encoding fitur.

dimulai dengan mendefinisikan fungsi transformed_fn() untuk mengubah data sesuai dengan skema dan kemudian menerapkan transformasi yang diperlukan. Setelah itu, kita akan mengemas transformasi ini dalam bentuk modul yang digunakan oleh komponen Transform di dalam pipeline.

### **Transform**

In [20]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

LABEL_KEY = "is_spam"
FEATURE_KEY = "message"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    print("Input keys:", inputs.keys())  # Debugging, tampilkan keys
    
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs


Overwriting modules/email_spam_transform.py


1.  `LABEL_KEY` dan `FEATURE_KEY`: Nama-nama fitur yang digunakan untuk label dan data input. Di sini, `LABEL_KEY` adalah kolom is_spam, yang akan diubah menjadi label (0 atau 1), dan `FEATURE_KEY` adalah kolom message, yang akan diproses.

2. `transformed_name()`: Menambahkan suffix "_xf" pada setiap nama fitur untuk menunjukkan bahwa fitur tersebut sudah diproses atau ditransformasikan. Misalnya, message akan menjadi message_xf setelah diproses.

3.  `preprocessing_fn()`:
    - Fungsi utama yang mengatur bagaimana fitur yang ada diubah.
    - Mengubah teks dalam message menjadi huruf kecil menggunakan tf.strings.lower().
    - Mengubah label is_spam menjadi tipe data integer (tf.int64) menggunakan tf.cast().

In [21]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:absl:Generating ephemeral wheel package for 'd:\\Coolyeah\\Dicoding\\Submission_Dicoding\\ml_ops\\Submission 1\\modules\\email_spam_transform.py' (including modules: ['email_spam_trainer', 'email_spam_transform', 'email_spam_tuner']).
INFO:absl:User module package has hash fingerprint version 8299438edde152d9ad3eec06d5778d31075b3360bf52bb53bec66d2441659103.
INFO:absl:Executing: ['d:\\Coolyeah\\Dicoding\\Submission_Dicoding\\ml_ops\\envs\\mlops-subs-1\\python.exe', 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpg5s8c5w5\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpwq8w7l8h', '--dist-dir', 'C:\\Users\\User\\AppData\\Local\\Temp\\tmp9zx027rk']
INFO:absl:Successfully built user code wheel distribution at 'shendi_teuku_maulana_efendi-pipeline\\email-spam-pipeline\\_wheels\\tfx_user_code_Transform-0.0+8299438edde152d9ad3eec06d5778d31075b3360bf52bb53bec66d2441659103-py3-none-any.whl'; target user module is 'email_spam_transform'.
INFO:ab

Input keys: dict_keys(['is_spam', 'message'])
Input keys: dict_keys(['is_spam', 'message'])
Input keys: dict_keys(['is_spam', 'message'])
Input keys: dict_keys(['is_spam', 'message'])
Input keys: dict_keys(['is_spam', 'message'])
Input keys: dict_keys(['is_spam', 'message'])
Input keys: dict_keys(['is_spam', 'message'])


INFO:absl:Feature is_spam has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature is_spam has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature is_spam has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature is_spam has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message has a shape dim {
  size: 1
}
. Setting to DenseTensor.


Input keys: dict_keys(['is_spam', 'message'])
INFO:tensorflow:Assets written to: shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\Transform\transform_graph\49\.temp_path\tftransform_tmp\9dfd131a2b014e33a5e5b5344aa5bcec\assets


INFO:tensorflow:Assets written to: shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\Transform\transform_graph\49\.temp_path\tftransform_tmp\9dfd131a2b014e33a5e5b5344aa5bcec\assets
INFO:absl:Writing fingerprint to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\Transform\transform_graph\49\.temp_path\tftransform_tmp\9dfd131a2b014e33a5e5b5344aa5bcec\fingerprint.pb
INFO:absl:Feature is_spam_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature is_spam_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 49
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## **Model Training**
Model Training dalam pipeline ini berfokus pada pembangunan model klasifikasi email untuk mendeteksi spam vs non-spam menggunakan TensorFlow dan TFX. Proses ini terdiri dari dua tahap utama: Trainer dan Tuner, yang bekerja secara berurutan untuk menghasilkan model dengan performa optimal.

### **Trainer**
Trainer bertugas untuk melatih model berdasarkan dataset yang telah diproses pada tahap sebelumnya dengan melatih model klasifikasi dengan Neural Network, di mana teks diubah menjadi token menggunakan TextVectorization lalu diproses melalui Embedding Layer dan Dense Layers. Model dikompilasi dengan Binary Crossentropy Loss dan Adam Optimizer, serta dilatih menggunakan dataset training dengan validasi. Digunakan Early Stopping dan ModelCheckpoint untuk menghindari overfitting dan menyimpan model terbaik sebelum diekspor untuk deployment.

In [22]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "is_spam"
FEATURE_KEY = "message"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }

Overwriting modules/email_spam_trainer.py


In [23]:
from tfx.proto import trainer_pb2
 
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

INFO:absl:Generating ephemeral wheel package for 'd:\\Coolyeah\\Dicoding\\Submission_Dicoding\\ml_ops\\Submission 1\\modules\\email_spam_trainer.py' (including modules: ['email_spam_trainer', 'email_spam_transform', 'email_spam_tuner']).
INFO:absl:User module package has hash fingerprint version 8299438edde152d9ad3eec06d5778d31075b3360bf52bb53bec66d2441659103.
INFO:absl:Executing: ['d:\\Coolyeah\\Dicoding\\Submission_Dicoding\\ml_ops\\envs\\mlops-subs-1\\python.exe', 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpsmbbhj9e\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\User\\AppData\\Local\\Temp\\tmp6p_7s5u9', '--dist-dir', 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpn0eeuiir']


INFO:absl:Successfully built user code wheel distribution at 'shendi_teuku_maulana_efendi-pipeline\\email-spam-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+8299438edde152d9ad3eec06d5778d31075b3360bf52bb53bec66d2441659103-py3-none-any.whl'; target user module is 'email_spam_trainer'.
INFO:absl:Full user module path is 'email_spam_trainer@shendi_teuku_maulana_efendi-pipeline\\email-spam-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+8299438edde152d9ad3eec06d5778d31075b3360bf52bb53bec66d2441659103-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:udf_utils.get_fn {'train_args': '{\n  "splits": [\n    "train"\n  ]\n}', 'eval_args': '{\n  "splits": [\n    "eval"\n  ]\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'email_spam_trainer@shendi_teuku_maulana_efendi-pipeline\\email-spam-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+829943

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.
INFO:absl:Feature is_spam_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message_xf has a shape dim {
  size: 1
}
. Setting to DenseTensor.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 message_xf (InputLayer)     [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                1088  

...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature is_spam has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature message has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Training complete. Model written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\Trainer\model\50\Format-Serving. ModelRun written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\Trainer\model_run\50
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 50
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### **Tuner**
Tahap Tuner melakukan pencarian hyperparameter terbaik untuk model menggunakan KerasTuner. Metode RandomSearch digunakan untuk mengeksplorasi berbagai kombinasi nilai hyperparameter seperti jumlah unit pada layer tersembunyi, learning rate, dan optimizer. Tujuannya adalah menemukan konfigurasi yang optimal untuk meningkatkan akurasi model tanpa overfitting. Hasil tuning kemudian digunakan untuk melatih model dengan parameter terbaik.

In [33]:
%%writefile {TUNER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any
from keras_tuner import RandomSearch
from tfx.components import Tuner, Trainer
from tfx.proto import trainer_pb2

LABEL_KEY = 'is_spam'
FEATURE_KEY = 'message'

# Rename transformed features
def transformed_name(key):
    return key + "_xf"

# Gzip file reader
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Dataset input function
def input_fn(file_pattern, tf_transform_output, num_epochs=2, batch_size=32):
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)
    )
    return dataset

VOCAB_SIZE = 2000
SEQUENCE_LENGTH = 500

vectorize_layer = layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

# Model builder for tuning
def model_builder(hp):
    embedding_dim = hp.Choice('embedding_dim', values=[32, 64])
    lstm_units = hp.Choice('lstm_units', values=[16, 32])
    dense_units = hp.Int('dense_units', min_value=32, max_value=128, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4])
    num_layers = hp.Int('num_layers', min_value=2, max_value=4, step=1)

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

    for _ in range(num_layers):
        x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

# Early stopping callback
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    mode='max',
    patience=1,
    verbose=1
)

# Tuner function
def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, num_epochs=1, batch_size=64)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output, num_epochs=1, batch_size=64)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)
        ]]
    )

    model_tuner = RandomSearch(
        hypermodel=model_builder,
        objective=kt.Objective('val_binary_accuracy', direction='max'),
        max_trials=1,
        executions_per_trial=1,
        directory=fn_args.working_dir,
        project_name='email_spam_tuner',
    )

    return TunerFnResult(
        tuner=model_tuner,
        fit_kwargs={
            'x': train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
            'callbacks': [early_stop_callback]
        }
    )


Overwriting modules/email_spam_tuner.py


In [ ]:
tuner = Tuner(
    module_file     = os.path.abspath(TUNER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(tuner)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'embedding_dim', 'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'lstm_units', 'default': 16, 'conditions': [], 'values': [16, 32], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'dense_units', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'dropout_rate', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'num_layers', 'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': 'linear'}}], 'values': {'embeddi

Trial 1 Complete [00h 00m 16s]
val_binary_accuracy: 0.9463459849357605

Best val_binary_accuracy So Far: 0.9463459849357605
Total elapsed time: 00h 00m 16s
Results summary
Results in shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\.temp\53\email_spam_tuner
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 16
dense_units: 128
dropout_rate: 0.1
learning_rate: 0.001
num_layers: 2
Score: 0.9463459849357605


ExecutionResult(
    component_id: Tuner
    execution_id: 53
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## **Model Analysis and Validation**
Setelah proses tuning selesai dan model terbaik telah dipilih, tahap berikutnya dalam pipeline adalah Model Analysis and Validation, yang terdiri dari tiga langkah utama:

### **Resolver**
- Resolver digunakan untuk memilih model terbaik dari kandidat model yang tersedia.
- Model dengan performa terbaik dipilih untuk diproses lebih lanjut sebelum di-push ke produksi.

In [41]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

INFO:absl:Running driver for Latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for Latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 57
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### **Evaluator**
- Komponen Evaluator digunakan untuk mengevaluasi model terlatih berdasarkan metrik yang telah ditentukan.
- Evaluator memanfaatkan TFX Model Evaluation untuk membandingkan model baru dengan model sebelumnya (jika ada).
- Evaluasi mencakup metrik seperti akurasi, precision, recall, dan AUC (Area Under Curve).

In [44]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='is_spam')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [45]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "FalsePositives"\n        },\n        {\n          "class_name": "TruePositives"\n        },\n        {\n          "class_name": "FalseNegatives"\n        },\n        {\n          "class_name": "TrueNegatives"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "is_sp

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\Evaluator\blessing\59.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 59
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [46]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

### Pusher
- Pusher bertanggung jawab untuk menyimpan dan men-deploy model yang telah divalidasi ke lingkungan produksi.
- Model yang lolos evaluasi akan didorong ke lokasi penyimpanan yang telah dikonfigurasi, seperti Google Cloud Storage atau TensorFlow Serving.
- Proses ini memastikan bahwa model yang akan digunakan di tahap 

In [50]:
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/email-spam-detection-model'))
 
)
 
interactive_context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1738600391
INFO:absl:Model written to serving path serving_model_dir/email-spam-detection-model\1738600391.
INFO:absl:Model pushed to shendi_teuku_maulana_efendi-pipeline\email-spam-pipeline\Pusher\pushed_model\61.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 61
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [51]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/email-spam-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1738600391'}]}
